In [9]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library("factoextra")
library("maptree")
library("ggpubr")
library("ica")
library(ICtest)

library(repr)
fileDir = "/home/beraslan/jovian-work/analysisSingle/"
geneCutOff=4
targetCutOff=15
nFactors=15

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa

Loading required package: cluster

Loading required package: rpart


Attaching package: ‘ggpubr’


The following object is masked from ‘package:cowplot’:

    get_legend


Loading required package: JADE

Loading required package: ICS

Loading required package: mvtnorm

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



In [10]:
selCoefs <- read.csv("./../TextFiles/SignificantCoefMatrix.csv", row.names = 1)
selCoefs <- t(selCoefs)
KOGenes <-  colnames(selCoefs)
KOGenes <- sapply(KOGenes, function(x){strsplit(x,"_")[[1]][2]})
colnames(selCoefs) <- KOGenes

In [11]:
head(selCoefs)

,Rack1,Utp15,Wdr43,Raf1,Rrp9,Uhrf1,Myc,Wdr3,Aamp,Dcaf13,⋯,Acaca,Trim45,Rnf135,Dcaf10,Traf7,Zmiz2,Klhl30,Wdr91,Efcab8,Wdr81
Eef1a1,-0.052,-0.147,-0.238,-0.125,-0.135,0.063,-0.397,0.028,-0.052,-0.101,⋯,0.158,-0.042,0.004,0.038,0.025,-0.066,-0.016,0.061,-0.033,0.085
Tpt1,-0.255,-0.096,-0.261,-0.026,-0.079,-0.218,-0.158,-0.159,-0.323,-0.190,⋯,0.104,0.112,0.068,0.024,0.099,0.095,0.119,0.104,0.119,0.050
Rps27,-0.289,-0.235,-0.094,-0.097,-0.104,-0.116,-0.167,-0.087,-0.081,-0.159,⋯,0.075,0.087,0.070,-0.003,0.061,0.024,0.060,0.028,0.077,0.035
Rplp0,-0.328,-0.076,-0.110,-0.130,-0.008,-0.134,-0.586,-0.017,-0.031,-0.064,⋯,0.113,0.017,-0.018,0.145,0.103,0.105,0.017,0.015,0.082,0.047
Denr,-0.140,-0.042,-0.027,-0.047,-0.070,-0.208,-0.079,-0.098,0.140,-0.104,⋯,0.086,0.103,-0.057,-0.118,0.049,0.149,0.198,0.042,0.070,0.017
Hmgb2,-0.078,0.024,-0.019,-0.001,-0.032,-0.083,-0.089,0.054,0.011,-0.039,⋯,0.029,-0.011,0.013,-0.003,0.018,0.045,0.183,0.007,0.124,0.020


In [12]:
#guideModulesN <- data.frame(read.csv("/home/beraslan/jovian-work/analysisSingle/ME_GuideModules_7_Modules.csv", row.names = 1))
#guideModulesN$GuideName <- sapply(guideModulesN$GuideName, function(x){strsplit(as.character(x),"_")[[1]][2]})

guideModulesN <- data.frame(read.csv("./../TextFiles/ME_GuideModules_leiden_6_Modules.csv"),  row.names = 1)
rownames(guideModulesN) <- guideModulesN$GuideName 

head(guideModulesN)

,GuideName,GuideGroup,GuideColor
,<chr>,<int>,<chr>
Aamp,Aamp,4,#aa40fc
Bop1,Bop1,4,#aa40fc
Cirh1a,Cirh1a,4,#aa40fc
Dcaf13,Dcaf13,4,#aa40fc
Grb2,Grb2,4,#aa40fc
Myc,Myc,4,#aa40fc


In [13]:
## This is for selecting training and validation sets in combo prediction
# allTestGuides = list()

# for(elem in unique(guideModulesN$GuideGroup)){
#     myTmp = guideModulesN[guideModulesN$GuideGroup == elem,]
#     k = sample(myTmp$GuideName, round(nrow(myTmp) *0.10, digits = 0), replace=FALSE)
#     allTestGuides = lappend(allTestGuides, k)
# }

# allTestGuides = unique(unlist(allTestGuides))
# allTrainingGuides = guideModulesN$GuideName[guideModulesN$GuideName %ni% allTestGuides]
# write.csv(allTestGuides, "./../TestGuides.csv")
# write.csv(allTrainingGuides, "./../TrainingGuides.csv")

In [14]:
evaluateFactorNo <- function(){
    allTestGuides = list()

    for(elem in unique(guideModulesN$GuideGroup)){
        myTmp = guideModulesN[guideModulesN$GuideGroup == elem,]
        k = sample(myTmp$GuideName, round(nrow(myTmp) *0.2, digits = 0), replace=FALSE)
        allTestGuides = lappend(allTestGuides, k)
    }

    allTestGuides = unique(unlist(allTestGuides))
    allTrainingGuides = guideModulesN$GuideName[guideModulesN$GuideName %ni% allTestGuides]
    selCoefsTest = data.frame(selCoefs[,allTestGuides])
    selCoefsTrain = data.frame(selCoefs[,allTrainingGuides])
    
    
    allExpVars = list()

    for(i in 1:30){
        k <- icaimax(selCoefsTrain, nc = i, center = T)
        mySelFac = data.frame(k$S)
        
        r2ForNfac= c()
        for(j in 1:ncol(selCoefsTest)){

            mySelFac$y = selCoefsTest[,j]
            k = lm("y ~ .", data= mySelFac)
            myPredict = predict(k, newdata = mySelFac)

            myR2 = computeR2(mySelFac$y, myPredict)
            r2ForNfac = c(r2ForNfac, myR2)
        }
        
        allExpVars = lappend(allExpVars, r2ForNfac)
    }
    
    
    allExpVars = data.frame(do.call(cbind, allExpVars))
    rownames(allExpVars) = colnames(selCoefsTest) 
    colnames(allExpVars) = paste0("NFac_",1:ncol(allExpVars))
    allExpVars$genes = rownames(allExpVars)
    allExpVarsMelted = melt(allExpVars)
    
    return(allExpVarsMelted)

}

In [15]:
hh <- evaluateFactorNo() 

for(i in 1:5){
    vv = evaluateFactorNo()
    
    hh = rbind(hh,  vv)
}


Using genes as id variables

Using genes as id variables

Using genes as id variables

Using genes as id variables

Using genes as id variables

Using genes as id variables



In [17]:
options(repr.plot.width=12, repr.plot.height=8)

pdf("./../SupplementaryFigures/S_5E.pdf", width=12, height=6)
ggplot(hh, aes(x=variable, y=value)) + 
     geom_violin(trim=FALSE)+
     geom_boxplot()+theme_minimal()+ 
     #stat_summary(fun.data = mean, geom = "text", size=6)+
        # stat_compare_means(method="t.test",paired = FALSE,
        #                         comparisons = list(c("Control", "KO")),
        #                         color="red", method.args = list(alternative = "less"),
        #                         label.y = c(30), aes(label = ..p.signif..))+
     #scale_fill_brewer(palette = "Pastel1")+
ylab("Explained variance in \n unseen perturbation responses (R^2)")+
theme(axis.text = element_text(size=15),
      axis.text.x = element_text(angle=90),
              axis.title =  element_text(size=16))+ylim(0,1) +xlab("Number of ICA factors") 

dev.off()

Warning message:
“Removed 1229 rows containing missing values (geom_violin).”


png 
  2